In [130]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm


In [131]:
num_classes = 5
learning_rate = 0.1
num_epochs = 10
in_channels = 3

In [132]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(76),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [133]:
train_data_path = '/Users/jesseseid/Jseid628/dataset/Hass Avocado Ripening Photographic Dataset/Avocado Ripeness Split /train'
test_data_path = '/Users/jesseseid/Jseid628/dataset/Hass Avocado Ripening Photographic Dataset/Avocado Ripeness Split /test'

# Create an ImageFolder dataset
# This will automatically find the classes (1, 2, 3, 4, 5) from subfolders
train_dataset = datasets.ImageFolder(root=train_data_path, transform=train_transforms)
test_dataset = datasets.ImageFolder(root=test_data_path, transform=test_transforms)

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)



In [134]:


#Neural Network Definition

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        #The pooling layer downsamples by half each time. Initially 28*28 -> 14*14 -> 7*7 due to the kernel size of 2 and stride 2.
        self.fc1 = None
        self.num_classes = num_classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
       
        x = x.reshape(x.shape[0], -1) 
        if self.fc1 is None:
            self.fc1 = nn.Linear(torch.flatten(x, 1).shape[1], self.num_classes)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x
        
model = CNN(in_channels=in_channels, num_classes=num_classes)

#Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

#Training
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        data, targets = data.to(device), targets.to(device)
        with torch.cuda.amp.autocast():
            outputs = model(data)
            loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if ((epoch % 1)== 0):
        print(f"Epoch {epoch} Loss: {loss.item()}")

100%|██████████| 736/736 [01:29<00:00,  8.24it/s]


Epoch 0 Loss: 1.408341884613037


100%|██████████| 736/736 [01:40<00:00,  7.33it/s]


Epoch 1 Loss: 1.1115576028823853


100%|██████████| 736/736 [01:48<00:00,  6.76it/s]


...

Epoch 7 Loss: 0.8984735012054443


100%|██████████| 736/736 [01:59<00:00,  6.15it/s]


Epoch 8 Loss: 1.1202272176742554


100%|██████████| 736/736 [02:04<00:00,  5.93it/s]

Epoch 9 Loss: 1.2480093240737915


In [137]:
#Test
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            outputs = model(x)
            #print(len(outputs)) #should be 64 (batch size)
            max_values, predictions = torch.max(outputs, 1)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    
        accuracy = float(num_correct) / float(num_samples) * 100
        print(f"Got {num_correct}/{num_samples} with accuracy {accuracy:.2f}%")

    model.train()

check_accuracy(test_loader, model)


Got 1548/2944 with accuracy 52.58%
